In [1]:
import io
import requests
from pymongo import MongoClient
from pprint import pprint
import re

In [2]:
uri = open('./../../mongo_uri.txt', 'r', encoding='utf-8').read()
client = MongoClient(uri)
print(client)
db = client.boletines_db
# pprint(db.boletines.find_one())

MongoClient(host=['clustershido-shard-00-02-uxkjr.gcp.mongodb.net:27017', 'clustershido-shard-00-01-uxkjr.gcp.mongodb.net:27017', 'clustershido-shard-00-00-uxkjr.gcp.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, retrywrites=True, w='majority', authsource='admin', replicaset='ClusterShido-shard-0', ssl=True)


In [3]:
boletines = db.boletines.find({}, {'boletin':1, '_id':0}).sort('fecha').limit(4)

In [4]:
for boletin in boletines:
    #Archivo a ser enviado
    files = {'file': io.StringIO(re.sub('\n', '\n\n', boletin['boletin']))}
    #Parámetros
    params = {'outf': 'tagged', 'format': 'json'}
    #Enviar petición
    url = "http://www.corpus.unam.mx/servicio-freeling/analyze.php"
    r = requests.post(url, files=files, params=params)
    #Convertir de formato json
    obj = r.json()

    #Ejemplo, obtener todos los lemas
    for sentence in obj:
        for word in sentence:
            print(word["token"])

El
candidato
de
la
coalición
Todos_por_México
a
la
Presidencia_de_la_República
,
José_Antonio_Meade
,
presentó
la
Coordinación_de_Atención_Ciudadana
y
el
Buzón_de_el_Candidato
.
A_través_de
estos
mecanismos
,
Meade
y
su
esposa
,
Juana_Cuevas
,
recabarán
las
propuestas
,
opiniones
,
inquietudes
y
necesidades
de
la
población
.
El
equipo
de
Atención_Ciudadana
estará
conformado
por
jóvenes
voluntarios
,
quienes
apoyan
el
proyecto
de
gobierno
de
el
candidato
de
la
coalición
Todos_por_México
y
desean
colaborar
en
la
construcción
de
un
país
más
justo
y
equitativo
,
donde
se
cierren
las
brechas
y
se
erradiquen
las
desigualdades
.
El
voluntariado
escuchará
a
las
personas
,
comprenderá
sus
problemáticas
y
ofrecerá
una
atención
integral
.
Acompañarán
a
el
abanderado
presidencial
y
su
esposa
a_lo_largo_y_ancho_de
la
República_Mexicana
.
Con
estas
acciones
,
Meade
refrendó
su
compromiso
para
encabezar
un
gobierno
cercano
a
la
gente
y
a
la
medida
de
sus
necesidades
.
El
candidato
presidencial
afirmó

In [21]:
test = "En un pronunciamiento especial, el candidato de la coalición Todos por México condenó enérgicamente los abusos —suscitados a consecuencia de la llamada política de “Tolerancia Cero”—, que incluyen la separación de niños y padres, así como el confinamiento de menores en jaulas. Meade llamó a la Organización de las Naciones Unidas y a Unicef a intervenir para detener los atropellos."
files = {'file': io.StringIO(test)}
#Parámetros
params = {'outf': 'tagged', 'format': 'json'}
#Enviar petición
url = "http://www.corpus.unam.mx/servicio-freeling/analyze.php"
r = requests.post(url, files=files, params=params)
#Convertir de formato json
obj = r.json()

#Ejemplo, obtener todos los lemas
for sentence in obj:
    for word in sentence:
        print(word["token"])

En
un
pronunciamiento
especial
,
el
candidato
de
la
coalición
Todos_por_México
condenó
enérgicamente
los
abusos
—
suscitados
a_consecuencia_de
la
llamada
política
de
“
Tolerancia
Cero
”
—
,
que
incluyen
la
separación
de
niños
y
padres
,
así_como
el
confinamiento
de
menores
en
jaulas
.
Meade
llamó
a
la
Organización_de_las_Naciones_Unidas
y
a
Unicef
a
intervenir
para
detener
los
atropellos
.


In [5]:
client.close()